## 주제
##### 앙상블 학습이 무엇인지 이해하고 다양한 앙상블 학습 알고리즘을 배우기

### 정형 데이터와 비정형 데이터
##### 정형 데이터
* CSV나 데이터베이스, 엑셀 형태로 되어있음
##### 비정형 데이터
* 데이터 베이스나 엑셀로 표현하기 힘듬
* 예시로 책의 글과 같은 텍스트 데이터, 디지털 카메라로 찍은 사진, 핸드폰으로 듣는 디지털 음악

정형데이터에 뛰어난 성과를 내는 알고리즘이 앙상블 학습임
### 랜덤 포레스트(Random Forest)
* 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용되고 있음
* 각 트리를 훈런하기 위한 데이터를 랜덤하게 만듬
* 우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듬(이때 한 샘플이 중복되어 추출될 수도 있음)

> 예를 들어 1000개의 샘플이 들어있는 가방에서 100개의 샘플을 뽑는다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣고, 이런 식으로 계속해서 100개를 가방에서 뽑으면 중복된
샘플을 뽑을 수 있음 


>----> 부트스트랩 샘플 (bootstrap sample)
> * 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만듬, 1000개의 샘플이 들어있는 가방에서 중복하여 1000개의 샘플을 뽑음
> * 정리: 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식을 의미
* 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택(4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용, 다만 회귀 모델에서는 전체 특성을 사용)
* 사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 훈련함
* 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 쓰고, 회귀일 때는 단순히 각 트리의 예측을 평균함
* 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42)

In [3]:
## 랜덤 포레스트는 결정 트리의 앙상블이기 때문에 디시젼 트리의 중요한 매개변수를 모두 제공함
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) ## 훈련 세트 과적합 

0.9973541965122431 0.8903229806766861


In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


observation:
5-1장에 중요도와 비교해보면 당도의 중요도는 감소하고 알코올 pH특성의 중요도가 조심 상승했음(이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문
그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음, 이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 됨)
* 랜덤 포레스트에 자체적으로 모델을 평가하는 점수를 얻을 수 있다.
* 랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정 트리를 훈련함(이때 부트스트랩 샘플에 포함되지 않고 남는 샘플도 있음 이것을 OOB(out of bag)샘플이라고 함)
--> 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. 마치 검증 세트의 역활을 함(이 점수를 얻으려면 oob_score 매개변수를 True로 지정해야 됨(기본값은 False))



In [5]:
## oob_score 출력
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8945545507023283


#### 엑스트라 트리(Extra Trees)
* 랜덤 포레스트와 매우 비슷하게 동작함, 기본적으로 100개이 결정 트리를 훈련함
* 랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수를 지원함, 또한 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용함
* 랜덤 포레스트와 엑스트라 트리의 차이점은 부트스트랩 샘플을 사용하지 않은 것, 즉 각 결정 트리를 만들 때 전체 훈련 세트를 사용함
* 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함 디시젼트리에서 splitter = 'random'과 같음
  하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는데 효과가 있음


In [6]:
## 엑스트라 모델 사용 성능은 랜덤 포레스트와 비슷한 결과를 얻었고, 이 예제는 특성이 많지 않아서 두 모델의 차이가 크지 않음
## 보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 됨(하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점임)
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [7]:
## 엑스트라 트리도 랜덤 포레스트와 마찬가지로 특성 중요도를 제공, 결과를 보면 엑스트라 트리도 결정 트리보다 당도에 대한 의존성이 작음
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


#### 그래디언트 부스팅(gradient boosting)
* 깊이가 얕은 결정 트리르 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법임
* gradinetBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용함
* 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음
* 4장에서 배운 경사 하강법을 사용하여 트리를 앙상블에 추가함 분류는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용함

##### 경사하강법
* 손실함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정으로 설명
* 이때 가장 낮은 곳을 찾아 내려오는 방법은 모델의 가중치와 절편을 조금씩 바꾸는 것
* 그레디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동함


In [8]:
## 과대적합이 사라짐 그래디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강함 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [9]:
## 결정 트리 개수를 500개로 5배나 늘렸지만 과대적합을 잘 억제하고 있음, 학습률의 기본값은 0.1임
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2,
                                random_state = 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
## 특성 중요도
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15881044 0.67988912 0.16130044]


##### 그래디언트 추가 옵션
* 트리 훈련에 사용할 훈련 세트의 비율을 정하는 subsample이 있음 이 매개변수의 기본값은 1.0임 
* 하지만 subsample이 1보다 작으면 훈련 세트의 일부를 사용함, 이는 마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 확률적 경사 하강법이나 미니배치 경사 하강법과 비슷함
* 일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있음 하지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느림
* 즉 그레디언트부스팅에는 n_jobs 매개변수가 없음 

In [13]:
## 히스토그램 기반 그레이디언트 부스팅 (Histogram_based Gradient Boosting) 과대적합을 잘 억제하면서 그레디언트 부스팅보다 조금 더 높은 성능을 제공함
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [15]:
## 특성 중요도를 계산하기 위해 permutation_importance()를 사용
## 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산함(훈련 세트뿐만 아니라 테스트 세트에도 적용 가능)
## n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정
## permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 특성 중요도, 평균, 표준편차를 나타냄(평균은 랜덤 포레스트와 비슷한 비율임을 알 수 있음)
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [16]:
## 테스트 결과를 보면 그레디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있음
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [17]:
## 랜덤 포레스트에서는 86%였고 1퍼센트 가량 오름
hgb.score(test_input, test_target)


0.8723076923076923

In [18]:
## XGB 모델 사용
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [20]:
## 정확도가 88퍼로 가장 높게 나옴
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000314 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000573 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757816 -> initscore=1.140744
[LightGBM] [Info] Start training from score 1.140744
[LightGBM] [Info] Number o

##### 간단한 정리
* 랜덤포레스트 : 부트스트랩 샘플 사용, 대표 앙상블 학습 알고리즘
* 엑스트라 트리 : 결정 트리의 노드를 랜덤하게 분할함
* 그레이디언트 부스팅 : 이전 트리의 손실을 보완하는 식으로 얕은 결정 트리를 연속하여 추가함
* 히스토그램 기반 그레이디언트 부스팅 : 훈련 데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄

## 정리
#### 개념
##### 앙상블 학습
* 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘
##### 랜덤 포레스트
* 대표적인 결정 트리 기반의 앙상블 학습 방법
* 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 만드는 것이 특징
##### 엑스트라 트리
* 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않음, 대신 랜덤하게 노드를 분할해 과대적합을 감소 시킴
##### 그레이디언트 부스팅 
* 그레이디언트 부스팅의 속도를 개선한 것이 히스토그램 기반 그레이디언트 부스팅이며 안정적인 결과와 높은 성능으로 매우 인기가 높음 

### 퀴즈
##### 1번
* 앙상블 학습이란 무엇일까요? 
##### 2번 
* 정형 데이터와 비정형 데이터의 차이점이 무엇일까요?
##### 3번
* n_jobs는 무엇을 하는 매개변수 일까요?
